In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


In [8]:
glove_file = '../../ml-toolbox-testing/dataset/glove.6B/glove.6B.300d.txt'
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')[:1]

embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(glove_file, encoding='latin'))


In [2]:
def build_vocab(texts):
    sentences = texts.progress_apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        if word in embeddings_index:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        elif word.capitalize() in embeddings_index:
            known_words[word] = embeddings_index[word.capitalize()]
            nb_known_words += vocab[word]
        elif word.lower() in embeddings_index:
            known_words[word] = embeddings_index[word.lower()]
            nb_known_words += vocab[word]
        elif word.upper() in embeddings_index:
            known_words[word] = embeddings_index[word.upper()]
            nb_known_words += vocab[word]
        else:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

def vocab_check_coverage(train, test):
    df = pd.concat([train, test]).reset_index(drop=True)
    
    vocab = build_vocab(df['text'])
    print("Glove : ")
    oov_glove = check_coverage(vocab, embeddings_index)
    oov_glove = {"oov_rate": len(oov_glove) / len(vocab), 'oov_words': oov_glove}
#     print("Paragram : ")
#     oov_paragram = check_coverage(vocab, embed_paragram)
#     oov_paragram = {"oov_rate": len(oov_paragram) / len(vocab), 'oov_words': oov_paragram}
#     print("FastText : ")
#     oov_fasttext = check_coverage(vocab, embed_fasttext)
#     oov_fasttext = {"oov_rate": len(oov_fasttext) / len(vocab), 'oov_words': oov_fasttext}
#     print("Google : ")
#     oov_google = check_coverage(vocab, embed_google)
#     oov_google = {"oov_rate": len(oov_google) / len(vocab), 'oov_words': oov_google}
    
    return oov_glove#, oov_paragram, oov_fasttext


In [15]:
print("Extracting GloVe embedding")
for word in list(embeddings_index.keys()):
    if word is "don't":
        print("found")


Extracting GloVe embedding


In [3]:
train = pd.read_csv('Dataset/train.csv')
test  = pd.read_csv('Dataset/test.csv')
# sub   = pd.read_csv('Dataset/Sample_Submission.csv')

train.shape, test.shape#, sub.shape

((5959, 3), (2553, 2))

In [4]:
train['Review Text'] = train['Review Text'].str.lower()
test['Review Text']  = test['Review Text'].str.lower()


In [5]:
contraction_mapping = {
    "Trump's" : 'trump is',"'cause": 'because','â€™': "'",',cause': 'because',';cause': 'because',"ain't": 'am not','ain,t': 'am not',
    'ain;t': 'am not','ain´t': 'am not','ain’t': 'am not',"aren't": 'are not','â€“': '-','â€œ':'"',
    'aren,t': 'are not','aren;t': 'are not','aren´t': 'are not','aren’t': 'are not',"can't": 'cannot',"can't've": 'cannot have','can,t': 'cannot','can,t,ve': 'cannot have',
    'can;t': 'cannot','can;t;ve': 'cannot have',
    'can´t': 'cannot','can´t´ve': 'cannot have','can’t': 'cannot','can’t’ve': 'cannot have',
    "could've": 'could have','could,ve': 'could have','could;ve': 'could have',"couldn't": 'could not',"couldn't've": 'could not have','couldn,t': 'could not','couldn,t,ve': 'could not have','couldn;t': 'could not',
    'couldn;t;ve': 'could not have','couldn´t': 'could not',
    'couldn´t´ve': 'could not have','couldn’t': 'could not','couldn’t’ve': 'could not have','could´ve': 'could have',
    'could’ve': 'could have',"didn't": 'did not','didn,t': 'did not','didn;t': 'did not','didn´t': 'did not',
    'didn’t': 'did not',"doesn't": 'does not','doesn,t': 'does not','doesn;t': 'does not','doesn´t': 'does not',
    'doesn’t': 'does not',"don't": 'do not','don,t': 'do not','don;t': 'do not','don´t': 'do not','don’t': 'do not',
    "hadn't": 'had not',"hadn't've": 'had not have','hadn,t': 'had not','hadn,t,ve': 'had not have','hadn;t': 'had not',
    'hadn;t;ve': 'had not have','hadn´t': 'had not','hadn´t´ve': 'had not have','hadn’t': 'had not','hadn’t’ve': 'had not have',"hasn't": 'has not','hasn,t': 'has not','hasn;t': 'has not','hasn´t': 'has not','hasn’t': 'has not',
    "haven't": 'have not','haven,t': 'have not','haven;t': 'have not','haven´t': 'have not','haven’t': 'have not',"he'd": 'he would',
    "he'd've": 'he would have',"he'll": 'he will',
    "he's": 'he is','he,d': 'he would','he,d,ve': 'he would have','he,ll': 'he will','he,s': 'he is','he;d': 'he would',
    'he;d;ve': 'he would have','he;ll': 'he will','he;s': 'he is','he´d': 'he would','he´d´ve': 'he would have','he´ll': 'he will',
    'he´s': 'he is','he’d': 'he would','he’d’ve': 'he would have','he’ll': 'he will','he’s': 'he is',"how'd": 'how did',"how'll": 'how will',
    "how's": 'how is','how,d': 'how did','how,ll': 'how will','how,s': 'how is','how;d': 'how did','how;ll': 'how will',
    'how;s': 'how is','how´d': 'how did','how´ll': 'how will','how´s': 'how is','how’d': 'how did','how’ll': 'how will',
    'how’s': 'how is',"i'd": 'i would',"i'll": 'i will',"i'm": 'i am',"i've": 'i have','i,d': 'i would','i,ll': 'i will',
    'i,m': 'i am','i,ve': 'i have','i;d': 'i would','i;ll': 'i will','i;m': 'i am','i;ve': 'i have',"isn't": 'is not',
    'isn,t': 'is not','isn;t': 'is not','isn´t': 'is not','isn’t': 'is not',"it'd": 'it would',"it'll": 'it will',"It's":'it is',
    "it's": 'it is','it,d': 'it would','it,ll': 'it will','it,s': 'it is','it;d': 'it would','it;ll': 'it will','it;s': 'it is','it´d': 'it would','it´ll': 'it will','it´s': 'it is',
    'it’d': 'it would','it’ll': 'it will','it’s': 'it is',
    'i´d': 'i would','i´ll': 'i will','i´m': 'i am','i´ve': 'i have','i’d': 'i would','i’ll': 'i will','i’m': 'i am',
    'i’ve': 'i have',"let's": 'let us','let,s': 'let us','let;s': 'let us','let´s': 'let us',
    'let’s': 'let us',"ma'am": 'madam','ma,am': 'madam','ma;am': 'madam',"mayn't": 'may not','mayn,t': 'may not','mayn;t': 'may not',
    'mayn´t': 'may not','mayn’t': 'may not','ma´am': 'madam','ma’am': 'madam',"might've": 'might have','might,ve': 'might have','might;ve': 'might have',"mightn't": 'might not','mightn,t': 'might not','mightn;t': 'might not','mightn´t': 'might not',
    'mightn’t': 'might not','might´ve': 'might have','might’ve': 'might have',"must've": 'must have','must,ve': 'must have','must;ve': 'must have',
    "mustn't": 'must not','mustn,t': 'must not','mustn;t': 'must not','mustn´t': 'must not','mustn’t': 'must not','must´ve': 'must have',
    'must’ve': 'must have',"needn't": 'need not','needn,t': 'need not','needn;t': 'need not','needn´t': 'need not','needn’t': 'need not',"oughtn't": 'ought not','oughtn,t': 'ought not','oughtn;t': 'ought not',
    'oughtn´t': 'ought not','oughtn’t': 'ought not',"sha'n't": 'shall not','sha,n,t': 'shall not','sha;n;t': 'shall not',"shan't": 'shall not',
    'shan,t': 'shall not','shan;t': 'shall not','shan´t': 'shall not','shan’t': 'shall not','sha´n´t': 'shall not','sha’n’t': 'shall not',
    "she'd": 'she would',"she'll": 'she will',"she's": 'she is','she,d': 'she would','she,ll': 'she will',
    'she,s': 'she is','she;d': 'she would','she;ll': 'she will','she;s': 'she is','she´d': 'she would','she´ll': 'she will',
    'she´s': 'she is','she’d': 'she would','she’ll': 'she will','she’s': 'she is',"should've": 'should have','should,ve': 'should have','should;ve': 'should have',
    "shouldn't": 'should not','shouldn,t': 'should not','shouldn;t': 'should not','shouldn´t': 'should not','shouldn’t': 'should not','should´ve': 'should have',
    'should’ve': 'should have',"that'd": 'that would',"that's": 'that is','that,d': 'that would','that,s': 'that is','that;d': 'that would',
    'that;s': 'that is','that´d': 'that would','that´s': 'that is','that’d': 'that would','that’s': 'that is',"there'd": 'there had',
    "there's": 'there is','there,d': 'there had','there,s': 'there is','there;d': 'there had','there;s': 'there is',
    'there´d': 'there had','there´s': 'there is','there’d': 'there had','there’s': 'there is',
    "they'd": 'they would',"they'll": 'they will',"they're": 'they are',"they've": 'they have',
    'they,d': 'they would','they,ll': 'they will','they,re': 'they are','they,ve': 'they have','they;d': 'they would','they;ll': 'they will','they;re': 'they are',
    'they;ve': 'they have','they´d': 'they would','they´ll': 'they will','they´re': 'they are','they´ve': 'they have','they’d': 'they would','they’ll': 'they will',
    'they’re': 'they are','they’ve': 'they have',"wasn't": 'was not','wasn,t': 'was not','wasn;t': 'was not','wasn´t': 'was not',
    'wasn’t': 'was not',"we'd": 'we would',"we'll": 'we will',"we're": 'we are',"we've": 'we have','we,d': 'we would','we,ll': 'we will',
    'we,re': 'we are','we,ve': 'we have','we;d': 'we would','we;ll': 'we will','we;re': 'we are','we;ve': 'we have',
    "weren't": 'were not','weren,t': 'were not','weren;t': 'were not','weren´t': 'were not','weren’t': 'were not','we´d': 'we would','we´ll': 'we will',
    'we´re': 'we are','we´ve': 'we have','we’d': 'we would','we’ll': 'we will','we’re': 'we are','we’ve': 'we have',"what'll": 'what will',"what're": 'what are',"what's": 'what is',
    "what've": 'what have','what,ll': 'what will','what,re': 'what are','what,s': 'what is','what,ve': 'what have','what;ll': 'what will','what;re': 'what are',
    'what;s': 'what is','what;ve': 'what have','what´ll': 'what will',
    'what´re': 'what are','what´s': 'what is','what´ve': 'what have','what’ll': 'what will','what’re': 'what are','what’s': 'what is',
    'what’ve': 'what have',"where'd": 'where did',"where's": 'where is','where,d': 'where did','where,s': 'where is','where;d': 'where did',
    'where;s': 'where is','where´d': 'where did','where´s': 'where is','where’d': 'where did','where’s': 'where is',
    "who'll": 'who will',"who's": 'who is','who,ll': 'who will','who,s': 'who is','who;ll': 'who will','who;s': 'who is',
    'who´ll': 'who will','who´s': 'who is','who’ll': 'who will','who’s': 'who is',"won't": 'will not','won,t': 'will not','won;t': 'will not',
    'won´t': 'will not','won’t': 'will not',"wouldn't": 'would not','wouldn,t': 'would not','wouldn;t': 'would not','wouldn´t': 'would not',
    'wouldn’t': 'would not',"you'd": 'you would',"you'll": 'you will',"you're": 'you are','you,d': 'you would','you,ll': 'you will',
    'you,re': 'you are','you;d': 'you would','you;ll': 'you will',
    'you;re': 'you are','you´d': 'you would','you´ll': 'you will','you´re': 'you are','you’d': 'you would','you’ll': 'you will','you’re': 'you are',
    '´cause': 'because','’cause': 'because',"you've": "you have","could'nt": 'could not',
    "havn't": 'have not',"here’s": "here is",'i""m': 'i am',"i'am": 'i am',"i'l": "i will","i'v": 'i have',"wan't": 'want',"was'nt": "was not","who'd": "who would",
    "who're": "who are","who've": "who have","why'd": "why would","would've": "would have","y'all": "you all","y'know": "you know","you.i": "you i",
    "your'e": "you are","arn't": "are not","agains't": "against","c'mon": "common","doens't": "does not",'don""t': "do not","dosen't": "does not",
    "dosn't": "does not","shoudn't": "should not","that'll": "that will","there'll": "there will","there're": "there are",
    "this'll": "this all","u're": "you are", "ya'll": "you all","you'r": "you are","you’ve": "you have","d'int": "did not","did'nt": "did not","din't": "did not","dont't": "do not","gov't": "government",
    "i'ma": "i am","is'nt": "is not","‘I":'I',
    'ᴀɴᴅ':'and','ᴛʜᴇ':'the','ʜᴏᴍᴇ':'home','ᴜᴘ':'up','ʙʏ':'by','ᴀᴛ':'at','…and':'and','civilbeat':'civil beat',\
    'TrumpCare':'Trump care','Trumpcare':'Trump care', 'OBAMAcare':'Obama care','ᴄʜᴇᴄᴋ':'check','ғᴏʀ':'for','ᴛʜɪs':'this','ᴄᴏᴍᴘᴜᴛᴇʀ':'computer',\
    'ᴍᴏɴᴛʜ':'month','ᴡᴏʀᴋɪɴɢ':'working','ᴊᴏʙ':'job','ғʀᴏᴍ':'from','Sᴛᴀʀᴛ':'start','gubmit':'submit','CO₂':'carbon dioxide','ғɪʀsᴛ':'first',\
    'ᴇɴᴅ':'end','ᴄᴀɴ':'can','ʜᴀᴠᴇ':'have','ᴛᴏ':'to','ʟɪɴᴋ':'link','ᴏғ':'of','ʜᴏᴜʀʟʏ':'hourly','ᴡᴇᴇᴋ':'week','ᴇɴᴅ':'end','ᴇxᴛʀᴀ':'extra',\
    'Gʀᴇᴀᴛ':'great','sᴛᴜᴅᴇɴᴛs':'student','sᴛᴀʏ':'stay','ᴍᴏᴍs':'mother','ᴏʀ':'or','ᴀɴʏᴏɴᴇ':'anyone','ɴᴇᴇᴅɪɴɢ':'needing','ᴀɴ':'an','ɪɴᴄᴏᴍᴇ':'income',\
    'ʀᴇʟɪᴀʙʟᴇ':'reliable','ғɪʀsᴛ':'first','ʏᴏᴜʀ':'your','sɪɢɴɪɴɢ':'signing','ʙᴏᴛᴛᴏᴍ':'bottom','ғᴏʟʟᴏᴡɪɴɢ':'following','Mᴀᴋᴇ':'make',\
    'ᴄᴏɴɴᴇᴄᴛɪᴏɴ':'connection','ɪɴᴛᴇʀɴᴇᴛ':'internet','financialpost':'financial post', 'ʜaᴠᴇ':' have ', 'ᴄaɴ':' can ', 'Maᴋᴇ':' make ', 'ʀᴇʟɪaʙʟᴇ':' reliable ', 'ɴᴇᴇᴅ':' need ',
    'ᴏɴʟʏ':' only ', 'ᴇxᴛʀa':' extra ', 'aɴ':' an ', 'aɴʏᴏɴᴇ':' anyone ', 'sᴛaʏ':' stay ', 'Sᴛaʀᴛ':' start', 'SHOPO':'shop',
    }


def correct_contraction(x, dic):
    for word in dic.keys():
        if word in x:
            x = x.replace(word, dic[word])
    return x

train['Review Text'] = train['Review Text'].progress_apply(lambda x: correct_contraction(x, contraction_mapping))
test['Review Text']  = test['Review Text'].progress_apply(lambda x: correct_contraction(x, contraction_mapping))


100%|██████████| 2553/2553 [00:00<00:00, 11254.94it/s]


In [6]:
train.rename(columns={'Review Text':'text', 'Review Title':'title'}, inplace=True)
test.rename(columns={'Review Text':'text', 'Review Title':'title'}, inplace=True)
test.columns

Index(['text', 'title'], dtype='object')

In [9]:
import os,operator

extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&',
    '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
    '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',
    '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”',
    '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾',
    '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼',
    '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
    'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»',
    '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '«', '»', '´', 'º', '¾', '¡', '§', '£', '₤']


import string
my_punct = list(string.punctuation)
all_punct = list(set(my_punct + extra_punct))

special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text

def clean_special_punctuations(text):
    for punc in special_punc_mappings:
        if punc in text:
            text = text.replace(punc, special_punc_mappings[punc])
    # remove_diacritics don´t' ->  'don t'
    #text = remove_diacritics(text)
    return text



def preprocess(text):
    text = spacing_punctuation(text)
    text = clean_special_punctuations(text)
    return text


train["text"] = train["text"].progress_apply(preprocess)
test["text"]  = test["text"].progress_apply(preprocess)

train['text'] = train['text'].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')
test['text']  = test['text'].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')

train['text'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
test['text'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

train['text'].replace({'  ':' '}, regex=True, inplace=True)
test['text'].replace({'  ':' '}, regex=True, inplace=True)

100%|██████████| 2553/2553 [00:00<00:00, 99763.90it/s]


In [11]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train['text'] = train['text'].progress_apply(clean_text)
test['text']  = test['text'].progress_apply(clean_text)


100%|██████████| 2553/2553 [00:00<00:00, 22637.21it/s]

In [41]:
oov_glove = vocab_check_coverage(train, test)

/home/ankish/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

100%|██████████| 8512/8512 [00:00<00:00, 203193.53it/s]

Glove : 
Found embeddings for 91.36% of vocab
Found embeddings for  99.55% of all text


In [42]:
oov_glove['oov_words'][:50]


[('solimo', 101),
 ('prenatals', 88),
 ('vitafusion', 52),
 ('softgels', 42),
 ('cetaphil', 42),
 ('muscletech', 35),
 ('softgel', 34),
 ('preworkout', 29),
 ('bcaa', 29),
 ('revly', 22),
 ('500mg', 15),
 ('idk', 15),
 ('gummie', 14),
 ('bcaas', 13),
 ('videoid', 13),
 ('3mg', 13),
 ('5mg', 13),
 ('swallowable', 13),
 ('1mg', 12),
 ('cellucor', 12),
 ('methylcobalamin', 11),
 ('1000mg', 11),
 ('naturewise', 10),
 ('protien', 10),
 ('mykind', 10),
 ('preworkouts', 9),
 ('shakeology', 9),
 ('asorbic', 9),
 ('250mg', 9),
 ('isopure', 8),
 ('125mg', 8),
 ('wholefoods', 8),
 ('petrolatum', 8),
 ('nutribullet', 7),
 ('phenoxyethanol', 7),
 ('naturemade', 7),
 ('stawberry', 7),
 ('octinoxate', 6),
 ('oxybenzone', 6),
 ('dimethicone', 6),
 ('natrue', 6),
 ('fishiness', 6),
 ('5lbs', 6),
 ('bleh', 6),
 ('rxbars', 6),
 ('benedryl', 6),
 ('lasership', 6),
 ('solimno', 6),
 ('100mg', 6),
 ('burmannii', 6)]

In [12]:
import re

train['text'] = train['text'].str.replace('# ','')
train['text'] = train['text'].str.replace(' - ','')
train['text'] = train['text'].str.replace(' : ','')

test['text'] = test['text'].str.replace('#','')
test['text'] = test['text'].str.replace(' - ','')
test['text'] = test['text'].str.replace(' : ','')

In [ ]:
# def splitting(data):
#     new=[]
#     for sentences in data:
#         yes = sentences.split(". ")
#         new.append(yes) 
#     return new
    
# train['new_text'] = splitting(train['text'])
# test['new_text'] = splitting(test['text'])


# def cleaning1(data):
#     new_text=[]
#     for sentences in data:
#         matching = [s for s in sentences if 'reply posted' not in s]
#         new_text.append(matching)
        
#     return new_text

# train['new_text'] = cleaning1(train['new_text'])
# test['new_text'] = cleaning1(test['new_text'])


# def cleaning2(data):
#     new_text=[]
#     for sentences in data:
#         matching = [s for s in sentences if 'help center' not in s]
#         new_text.append(matching)
        
#     return new_text

# train['new_text'] = cleaning2(train['new_text'])
# test['new_text'] = cleaning2(test['new_text'])


# def cleaning3(data):
#     new_text=[]
    
#     for sentences in data:
#         if(len(sentences)>1):
#             matching = [s for s in sentences if len(s) >= 15]
#             new_text.append(matching)
            
#         else:
#             matching = [s for s in sentences if len(s) >= 2]
#             new_text.append(matching)
        
#     return new_text

# train['new_text'] = cleaning3(train['new_text'])
# test['new_text'] = cleaning3(test['new_text'])

# def don(d):
#     n=[]
#     for s in d:
#         res = ".".join(s)
#         n.append(res)
#     return n

# train['text'] = don(train['new_text'])
# test['text']  = don(test['new_text'])


In [15]:
def clean_text(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

train['text'] = train['text'].progress_apply(clean_text)
test['text']  = test['text'].progress_apply(clean_text)




  0%|          | 0/5959 [00:00<?, ?it/s]

100%|██████████| 5959/5959 [00:00<00:00, 98919.37it/s]

  0%|          | 0/2553 [00:00<?, ?it/s]

100%|██████████| 2553/2553 [00:00<00:00, 95665.74it/s]

In [16]:
# os.makedirs('new_dataset')
train.to_csv('new_dataset/train1.csv', index=None)
test.to_csv('new_dataset/test1.csv', index=None)
train.shape, test.shape

((5959, 3), (2553, 2))

In [19]:
import gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF

def tfidf_feature(train, test, col_name, min_df=3, analyzer='word', 
                  token_pattern=r'\w{1,}', ngram=3, stopwords='english', 
                  n_component=120, decom_flag=False, which_method='svd', 
                  max_features=None):
    """return tfidf feature
    Args:
        train, test: dataframe
        col_name: column name of text feature
        min_df: if Int, then it represent count of the minimum words in corpus (remove very rare word)
        analyzer: [‘word’, ‘char’]
        ngram: max range of ngram
        token_pattern: [using: r'\w{1,}'] [by default: '(?u)\b\w\w+\b']
        stopwords: ['english' or customized by remove specific words]
        n_component: n_component of svd feature transform
        decom_flag: Wheteher to run svd/nmf on top of that or not (by default: False)
        which_method: which to run [svd or nmf] on top of tfidf (by default: False)
        max_features: max no of features to keep, based on frequency. It will keep words with higher freq
    return:
        Transformed feature space of the text data, as well as tfidf function instance
        if svd_flag== True : train_tf, test_tf, tfv, svd
        else : train_tf, test_tf, tfv
    example:
        train_tfv, test_tfv, tfv = tfidf_feature(X_train, X_test, ['text'], min_df=3)
        train_svd, test_svd, complete_tfv, tfv, svd = tfidf_feature(X_train, X_test, ['text'], 
            min_df=3, svd_component=3, svd_flag=True)

    """
    tfv = TfidfVectorizer(min_df=min_df,  max_features=max_features, 
                strip_accents='unicode', analyzer=analyzer, max_df=1.0, 
                token_pattern=token_pattern, ngram_range=(1, ngram), 
                use_idf=1, smooth_idf=1, sublinear_tf=1,
                stop_words = stopwords)

    complete_df = pd.concat([train[col_name], test[col_name]], axis=0)
#         return complete_df
#         print(complete_df.shape, complete_df.columns)

    tfv.fit(list(complete_df[:].values))

    if decom_flag is False:
        train_tfv =  tfv.transform(train[col_name].values.ravel()) 
        test_tfv  = tfv.transform(test[col_name].values.ravel())

        del complete_df
        gc.collect()
        return train_tfv, test_tfv, tfv
    else:
        complete_tfv = tfv.transform(complete_df[:].values.ravel())
        
        if which_method is 'svd':
            svd = TruncatedSVD(n_components=n_component)
            svd.fit(complete_tfv)
            complete_dec = svd.transform(complete_tfv)
        else:
            nmf = NMF(n_components=n_component, random_state=1234, alpha=0, l1_ratio=0)
            nmf.fit(complete_tfv)            
            complete_dec = nmf.fit_transform(complete_tfv)            
        
        
        complete_dec = pd.DataFrame(data=complete_dec)
        complete_dec.columns = [which_method+'_'+str(i) for i in range(n_component)]

        train_dec = complete_dec.iloc[:train.shape[0]]
        test_dec = complete_dec.iloc[train.shape[0]:].reset_index(drop=True)

        del complete_dec, complete_df
        gc.collect()
        return train_dec, test_dec, complete_tfv, tfv

def countvect_feature(train, test, col_name, min_df=3, analyzer='word', token_pattern=r'\w{1,}', ngram=3, stopwords='english', max_features=None):
    """return CountVectorizer feature
    Args:
        train, test: dataset
        col_name: columns name of the text feature
        min_df: if Int, then it represent count of the minimum words in corpus (remove very rare word)
        analyzer: [‘word’, ‘char’]
        ngram: max range of ngram
        token_pattern: [using: r'\w{1,}'] [by default: '(?u)\b\w\w+\b']
        stopwords: ['english' or customized by remove specific words]
        max_features: max no of features to keep, based on frequency. It will keep words with higher freq
    return:
        Count feature space of the text data, as well as its function instance
    """
    ctv = CountVectorizer(min_df=min_df,  max_features=max_features, 
                strip_accents='unicode', analyzer=analyzer, 
                token_pattern=token_pattern, ngram_range=(1, ngram), 
                stop_words = stopwords)

    complete_df = pd.concat([train[col_name], test[col_name]], axis=0)
    ctv.fit(list(complete_df[:].values))

    train_tf =  ctv.transform(train[col_name].values.ravel()) 
    test_tf  = ctv.transform(test[col_name].values.ravel())

    del complete_df
    gc.collect()
    return train_tf, test_tf, ctv


In [30]:
out_tfidf1 = tfidf_feature(train, test, 'text', ngram=1)
out_tfidf2 = tfidf_feature(train, test, 'text', ngram=2)
out_tfidf3 = tfidf_feature(train, test, 'text', ngram=3)
out_tfidf4 = tfidf_feature(train, test, 'text', ngram=5)

out_vect1 = countvect_feature(train, test, 'text', ngram=1)
out_vect2 = countvect_feature(train, test, 'text', ngram=2)
out_vect3 = countvect_feature(train, test, 'text', ngram=3)
out_vect4 = countvect_feature(train, test, 'text', ngram=5)

print('tf-idf features: ', out_tfidf1[0].shape, 
     out_tfidf2[0].shape, out_tfidf3[0].shape, out_tfidf4[0].shape)
print('count-vect features: ', out_vect1[0].shape, 
     out_vect2[0].shape, out_vect3[0].shape, out_vect4[0].shape)


tf-idf features:  (5959, 5101) (5959, 24307) (5959, 40708) (5959, 70881)
count-vect features:  (5959, 5101) (5959, 24307) (5959, 40708) (5959, 70881)


## Multinomial bayes classifier

In [34]:
train['target'] = train['topic'].astype('category').cat.codes
train['target'] = train['target'].astype('int')
all_class = list(train['target'].unique())
print(len(all_class))

21


In [44]:
train[train['target'] == 1].shape, out_tfidf1.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf1[0], train['target'], stratify=train['target'], test_size=0.3
)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4171, 5101), (1788, 5101), (4171,), (1788,))

In [47]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, Y_train)
# pred = clf.predict(X_test)


(1788,)

In [58]:
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf1[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("tfidf1 : ", clf.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf2[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("tfidf2 : ", clf.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf3[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("tfidf3 : ", clf.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf4[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("tfidf4 : ", clf.score(X_test, Y_test))

print("=="*25)

from sklearn.naive_bayes import MultinomialNB
X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect1[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("count-vect1 : ", clf.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect2[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("count-vect2 : ", clf.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect3[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("count-vect3 : ", clf.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect4[0], train['target'], stratify=train['target'], test_size=0.3
)
clf = MultinomialNB().fit(X_train, Y_train)
print("count-vect4 : ", clf.score(X_test, Y_test))


tfidf1 :  0.36577181208053694
tfidf2 :  0.32606263982102907
tfidf3 :  0.3143176733780761
tfidf4 :  0.3070469798657718
count-vect1 :  0.4222595078299776
count-vect2 :  0.37472035794183445
count-vect3 :  0.3691275167785235
count-vect4 :  0.3635346756152125


In [90]:
logistic_reg = LogisticRegression(penalty='l2', dual=False, 
    C=0.01, fit_intercept=True, intercept_scaling=1, class_weight='balanced', 
    random_state=1234, max_iter=100, multi_class='warn', verbose=0, n_jobs=-1)
logistic_reg.fit(X_train, Y_train)
print(logistic_reg.score(X_test, Y_test))

0.48266219239373603


In [92]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.naive_bayes import MultinomialNB
X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf1[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("tfidf1 : ", logistic_reg.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf2[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("tfidf2 : ", logistic_reg.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf3[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("tfidf3 : ", logistic_reg.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_tfidf4[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("tfidf4 : ", logistic_reg.score(X_test, Y_test))

print("=="*25)

from sklearn.naive_bayes import MultinomialNB
X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect1[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("count-vect1 : ", logistic_reg.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect2[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("count-vect2 : ", logistic_reg.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect3[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("count-vect3 : ", logistic_reg.score(X_test, Y_test))

X_train, X_test, Y_train, Y_test = train_test_split(
    out_vect4[0], train['target'], stratify=train['target'], test_size=0.3
)
logistic_reg.fit(X_train, Y_train)
print("count-vect4 : ", logistic_reg.score(X_test, Y_test))


tfidf1 :  0.2595078299776286
tfidf2 :  0.2225950782997763
tfidf3 :  0.21756152125279643
tfidf4 :  0.2197986577181208
count-vect1 :  0.48210290827740493
count-vect2 :  0.48769574944071586
count-vect3 :  0.4949664429530201
count-vect4 :  0.46364653243847875


In [93]:
sub_df = pd.read_csv('Dataset/Sample_Submission.csv')
sub_df

,Review Text,Review Title,topic
0,"Not terrible, but not good. Tastes burnt and a...",Not my cup o’ joe,Burnt/ Over -roast
1,"I am so disappointed, it has no flavor, doesn'...","I am so disappointed, it has no flavor",Bad Flavor/Taste
2,"I recently gave up my daily 6 cups of coffee, ...",Flavor was dissapointing,Bitter
3,Meh. I've adored Peruvian coffee for 20 years....,Smooth but majorly bland. Won't repurchase.,NaN
4,Meh. I've adored Peruvian coffee for 20 years....,Smooth but majorly bland. Won't repurchase.,NaN


In [95]:
logistic_reg.fit(out_vect3[0], train['target'])
pred = logistic_reg.predict(out_vect3[1])
pred.shape

(2553,)

In [120]:
# def category_encoder(df)
test.drop('topic', axis=1, inplace=True)
test['target'] = pred
test.head()

,text,title,target
0,use chia seed protein shakes these tasted like...,Bad tast,1
1,use chia seed protein shakes these tasted like...,Bad tast,1
2,not waste your money,No change. No results.,11
3,use the book fortify your life tieraona low do...,"Good Vegan Choice, Poor Non Vegan Choice",10
4,use the book fortify your life tieraona low do...,"Good Vegan Choice, Poor Non Vegan Choice",10


In [121]:
test['topic'] = test['target'].apply(lambda x: class_mapping_reverse[str(x)])

In [124]:
test.drop('target', axis=1, inplace=True)
# os.makedirs('submission')
test.to_csv('submission/linear_model1.csv', index=None)

In [130]:
def get_mapping(df, col_name):
    cat_codes = df[col_name].astype('category')
    
    class_mapping = {}
    i = 0
    for col in cat_codes.cat.categories:
        class_mapping[col] = i
        i += 1
    
    class_mapping_reverse = {}
    for key, value in class_mapping.items():
        class_mapping_reverse[value] = key

    return class_mapping, class_mapping_reverse

In [132]:
class_mapping, class_mapping_reverse = get_mapping(train, 'topic')

In [80]:
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier

params = {}
params['alpha'] = 1
passive_agg = PassiveAggressiveClassifier(C=params['alpha'], early_stopping=False, validation_fraction=0.3, n_iter_no_change=5, shuffle=True, verbose=0, n_jobs=-1, random_state=1234, loss='hinge', class_weight='balanced', average=False, n_iter=None)
ridge_clf = RidgeClassifier(alpha=params['alpha'], fit_intercept=True, normalize=True, class_weight='balanced', random_state=1234)
logistic_reg = LogisticRegression(penalty='l2', dual=False, C=params['alpha'], fit_intercept=True, intercept_scaling=1, class_weight='balanced', random_state=1234, max_iter=100, multi_class='warn', verbose=0, n_jobs=-1)

In [81]:
ridge_clf.fit(X_train, Y_train)
print(ridge_clf.score(X_test, Y_test))
print("==============")

passive_agg.fit(X_train, Y_train)
print(passive_agg.score(X_test, Y_test))
print("==============")

logistic_reg.fit(X_train, Y_train)
print(logistic_reg.score(X_test, Y_test))
print("==============")


/home/ankish/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.07829977628635347


/home/ankish/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in PassiveAggressiveClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.37527964205816555


/home/ankish/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ankish/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ankish/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.42393736017897093


In [83]:
from sklearn.metrics import f1_score

for alpha in [0.01, 0.1, 0.5, 1, 5, 10]:
    logistic_reg = LogisticRegression(penalty='l2', dual=False, 
        C=alpha, fit_intercept=True, intercept_scaling=1, class_weight='balanced', 
        random_state=1234, max_iter=100, multi_class='warn', verbose=0, n_jobs=-1)
    logistic_reg.fit(X_train, Y_train)
    pred = logistic_reg.score(X_test, Y_test)
    print(pred)
#     print(f1_score(ts_y, pred, average='micro', sample_weight=None))
#     print(logistic_reg.score(ts_x, ts_y))
print("==============")

0.48266219239373603
0.4692393736017897
0.43512304250559286
0.42393736017897093
0.40380313199105144
0.39932885906040266


In [ ]:
for alpha in np.linspace(0.0001,0.1,10):
    logistic_reg = LogisticRegression(penalty='l2', dual=False, 
        C=alpha, fit_intercept=True, intercept_scaling=1, class_weight='balanced', 
        random_state=1234, max_iter=100, multi_class='warn', verbose=0, n_jobs=-1)
    logistic_reg.fit(X_train, Y_train)
    pred = logistic_reg.score(X_test, Y_test)
    print(alpha, " : ", pred)

In [ ]:
Usage: plot_document_classification_20newsgroups.py [options]

Options:
  -h, --help            show this help message and exit
  --report              Print a detailed classification report.
  --chi2_select=SELECT_CHI2
                        Select some number of features using a chi-squared
                        test
  --confusion_matrix    Print the confusion matrix.
  --top10               Print ten most discriminative terms per class for
                        every classifier.
  --all_categories      Whether to use all categories or not.
  --use_hashing         Use a hashing vectorizer.
  --n_features=N_FEATURES
                        n_features when using the hashing vectorizer.
  --filtered            Remove newsgroup information that is easily overfit:
                        headers, signatures, and quoting.

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
data loaded
2034 documents - 3.980MB (training set)
1353 documents - 2.867MB (test set)
4 categories

Extracting features from the training data using a sparse vectorizer
done in 0.412178s at 9.655MB/s
n_samples: 2034, n_features: 33809

Extracting features from the test data using the same vectorizer
done in 0.351330s at 8.162MB/s
n_samples: 1353, n_features: 33809

================================================================================
Ridge Classifier
________________________________________________________________________________
Training:
RidgeClassifier(solver='sag', tol=0.01)
train time: 0.132s
test time:  0.001s
accuracy:   0.896
dimensionality: 33809
density: 1.000000


================================================================================
Perceptron
________________________________________________________________________________
Training:
Perceptron(max_iter=50)
train time: 0.017s
test time:  0.002s
accuracy:   0.888
dimensionality: 33809
density: 0.255302


================================================================================
Passive-Aggressive
________________________________________________________________________________
Training:
PassiveAggressiveClassifier(max_iter=50)
train time: 0.031s
test time:  0.002s
accuracy:   0.904
dimensionality: 33809
density: 0.694674


================================================================================
kNN
________________________________________________________________________________
Training:
KNeighborsClassifier(n_neighbors=10)
train time: 0.002s
test time:  0.317s
accuracy:   0.858

================================================================================
Random forest
________________________________________________________________________________
Training:
RandomForestClassifier(n_estimators=100)
train time: 1.671s
test time:  0.071s
accuracy:   0.840

================================================================================
L2 penalty
________________________________________________________________________________
Training:
LinearSVC(dual=False, tol=0.001)
train time: 0.145s
test time:  0.002s
accuracy:   0.900
dimensionality: 33809
density: 1.000000


________________________________________________________________________________
Training:
SGDClassifier(max_iter=50)
train time: 0.030s
test time:  0.002s
accuracy:   0.902
dimensionality: 33809
density: 0.579380


================================================================================
L1 penalty
________________________________________________________________________________
Training:
LinearSVC(dual=False, penalty='l1', tol=0.001)
train time: 0.301s
test time:  0.002s
accuracy:   0.873
dimensionality: 33809
density: 0.005553


________________________________________________________________________________
Training:
SGDClassifier(max_iter=50, penalty='l1')
train time: 0.093s
test time:  0.002s
accuracy:   0.887
dimensionality: 33809
density: 0.022901


================================================================================
Elastic-Net penalty
________________________________________________________________________________
Training:
SGDClassifier(max_iter=50, penalty='elasticnet')
train time: 0.252s
test time:  0.002s
accuracy:   0.899
dimensionality: 33809
density: 0.187472


================================================================================
NearestCentroid (aka Rocchio classifier)
________________________________________________________________________________
Training:
NearestCentroid()
train time: 0.004s
test time:  0.002s
accuracy:   0.855

================================================================================
Naive Bayes
________________________________________________________________________________
Training:
MultinomialNB(alpha=0.01)
train time: 0.003s
test time:  0.001s
accuracy:   0.899
dimensionality: 33809
density: 1.000000


________________________________________________________________________________
Training:
BernoulliNB(alpha=0.01)
train time: 0.004s
test time:  0.003s
accuracy:   0.884
dimensionality: 33809
density: 1.000000


________________________________________________________________________________
Training:
ComplementNB(alpha=0.1)
train time: 0.004s
test time:  0.001s
accuracy:   0.911
dimensionality: 33809
density: 1.000000


================================================================================
LinearSVC with L1-based feature selection
________________________________________________________________________________
Training:
Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False, penalty='l1',
                                                     tol=0.001))),
                ('classification', LinearSVC())])
train time: 0.252s
test time:  0.002s
accuracy:   0.880

In [22]:
# tf_train[i] # feature count vector for training case i
# y_train[i] # label for training case i

# The count vectors are defined as:

# p = sum of all feature count vectors with label 1

# p = tf_train[y_train==1].sum(0) + 1

# q = sum of all feature count vectors with label 0

# q = tf_train[y_train==0].sum(0) + 1

# Notice that we add 1 to both count vectors to ensure that every token appear at least one time in each class.

# The log-count ratio r is:

# r = np.log((p/p.sum()) / (q/q.sum()))

# And b:

# b = np.log(len(p) / len(q))

# Just the ratio of number of positive and negative training cases.

In [ ]:
pre_preds = tf_test @ r.T + b
preds = pre_preds.T > 0
accuracy = (preds == y_test).mean()

In [65]:
out_tfidf1[0][train['target'] == 5].sum(0)

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [66]:
out_tfidf1[0][train['target'] == 4].sum(0)

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [67]:
np.sum(np.equal(out_tfidf1[0][train['target'] == 5].sum(0), 
                out_tfidf1[0][train['target'] == 3].sum(0)))

5101

In [68]:
out_tfidf1[0][train['target'] == 5].shape

(5959, 5101)

In [69]:
out_tfidf1[0][train['target'] == 2].shape

(5959, 5101)

In [75]:
idx = train[train['target'] == 2].index

In [78]:
out_tfidf1[0][idx].sum(0)

matrix([[0.        , 0.23786109, 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [46]:
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

# train['text1'] = train['text'].progress_apply(lemmatization)
# test['text1']  = test['text'].progress_apply(lemmatization)


In [47]:
df = train.append(test, ignore_index=True)
df['text1'] = df['text'].apply(lemmatization)

/home/ankish/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


KeyboardInterrupt: 

In [52]:
df[:5]['text'].apply(lemmatization)

0    [d, i, d,  , n, o, t, h, i, n, g,  , f, o, r, ...
1    [d, i, d,  , n, o, t, h, i, n, g,  , f, o, r, ...
2    [ , h, a, v, e,  , b, o, u, g, h, t,  , t, h, ...
3    [g, a, v, e,  , m, e,  , a, n,  , a, l, l, e, ...
4    [t, h, e, s, e,  , d, o,  , n, o, t,  , c, o, ...
Name: text, dtype: object

In [51]:
df1

,text,title,topic
0,"do nothing for -PRON- , do not help lose even ...",Useless,shipment and delivery
1,"do nothing for -PRON- , do not help lose even ...",Useless,not effective
2,have buy these bag and immediately open one ...,trash ! ! ! Do not buy these bag -PRON- ’ a wa...,Customer Service
3,give -PRON- an allergic reaction on -PRON- fac...,Do not recommend,allergic
4,these do not compare to the name brand wipe . ...,can not tackle big mess,texture


In [74]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]



In [56]:
df1['text'].apply(lemmatize_text)

0    [did, nothing, for, me, ,, did, not, help, los...
1    [did, nothing, for, me, ,, did, not, help, los...
2    [have, bought, these, bag, and, immediately, o...
3    [gave, me, an, allergic, reaction, on, my, fac...
4    [these, do, not, compare, to, the, name, brand...
5    [these, do, not, compare, to, the, name, brand...
6    [these, do, not, compare, to, the, name, brand...
7                                           [no, good]
8    [these, are, extremely, hard, to, swallow, ., ...
9    [first, of, all, the, style, am, leaving, revi...
Name: text, dtype: object

In [ ]:
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(text):
    return []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output


In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [76]:
# word_tokenize(df1['text'][4])

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

porter=PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [63]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

porter=PorterStemmer()
porter.stem('helping')

'help'

In [73]:
for word in df1['text'][1].split(" "):
    print(porter.stem(word))

did
noth
for
me
,
did
not
help
lost
even
with
work
out
and
eat
healthi
.
did
not
curb
appetit
or
anyth
.



In [78]:
import gc
del df1, df
gc.collect()

865

In [79]:
df = train.append(test, ignore_index=True)
df.shape

/home/ankish/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


(8512, 3)

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

cv = CountVectorizer(max_df=0.95,min_df=2,stop_words='english')
term_matrix = cv.fit_transform(df['text'])
# print(term_matrix)

In [104]:
lda = LatentDirichletAllocation(n_components=5, n_jobs=4)
lda.fit(term_matrix)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=4, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [105]:
gc.collect()

366

In [106]:
lda.components_[0]

array([0.20114701, 6.74804017, 0.20373605, ..., 2.194052  , 0.46571526,
       0.20000209])

In [107]:
topic = lda.components_[0]
top_words_indices = topic.argsort()[-10:]
for index in top_words_indices:
    print(cv.get_feature_names()[index])

seal
br
opened
time
just
box
broken
product
brand
bottle


In [108]:
topic_word_dict = {}
for index,topic in enumerate(lda.components_):
    words = [cv.get_feature_names()[i] for i in topic.argsort()[-10:]]
    topic_word_dict[index] = words
    print('Top words for topic {}'.format(index))
    print(words)
    print('-'*120)


Top words for topic 0
['seal', 'br', 'opened', 'time', 'just', 'box', 'broken', 'product', 'brand', 'bottle']
------------------------------------------------------------------------------------------------------------------------
Top words for topic 1
['bad', 'good', 'product', 'just', 'stomach', 'work', 'day', 'did', 'taking', 'br']
------------------------------------------------------------------------------------------------------------------------
Top words for topic 2
['br', 'bad', 'money', 'item', 'order', 'brand', 'ordered', 'did', 'received', 'product']
------------------------------------------------------------------------------------------------------------------------
Top words for topic 3
['good', 'does', 'powder', 'bad', 'just', 'br', 'protein', 'flavor', 'like', 'taste']
------------------------------------------------------------------------------------------------------------------------
Top words for topic 4
['pills', 'brand', 'did', 'product', 'gummy', 'good', 'vit

In [ ]:

topics = lda.transform(term_matrix)
data['topic'] = topics.argmax(axis=1)


def assign_topics(row):
    topic = row['topic']
    words = topic_word_dict[topic]

    return words


data['topic words'] = data.apply(assign_topics,axis=1)
print(data.head())

